In [1]:
import argparse
import bleu
import weighted_ngram_match
import syntax_match
import dataflow_match
import json
import re
from rouge import Rouge
from tree_sitter import Language, Parser

def read_json(name):
    with open(name,'r') as f:
        json_file = json.load(f)
    return json_file
def write_json(file,path):
    with open(path,'w') as f:
        json.dump(file,f)
def read_txt_last(fname):
    with open(fname, 'r', encoding='utf-8') as f: 
        lines = f.readlines()  
        return lines[-1]

ImportError: cannot import name 'DFG_python'

In [2]:
lang = 'python'
params = '0.1,0.4,0.1,0.4'
alpha,beta,gamma,theta = [float(x) for x in params.split(',')]

In [3]:
code = 'df.columns = [\'A\' madeupword0087]'
def remove_madeupword(code):
    item_list = re.split(r'(madeupword\d{4})',code)
    item_effective = []
    for s in item_list:
        if 'madeupword' in s:
            continue
        else:
            item_effective.append(s)
    return ''.join(item_effective)
#print(item_effective)
print(remove_madeupword(code))

df.columns = ['A' ]


In [4]:
post_replace = {
    "Ġ": " ",
    "Ċ": "\n",
    "ĉ": "\t",
    'madeupword0001': '\'jupyter_string\''
}
def remove_madeupword(code):
    item_list = re.split(r'(madeupword\d{4})',code)
    item_effective = []
    for s in item_list:
        if 'madeupword' in s:
            continue
        else:
            item_effective.append(s)
    return ''.join(item_effective)

def decode(hyp):
    hyp = ''.join(hyp)
    for s, t in post_replace.items():
        hyp = hyp.replace(s, t)
    hyp = remove_madeupword(hyp)
    return hyp
def decode_string(hyp):
    hyp = ''.join(hyp.split())
    for s, t in post_replace.items():
        hyp = hyp.replace(s, t)
    hyp = remove_madeupword(hyp)
    return hyp

In [10]:
def get_devdiv_codebleu(prefix):
    hypothesis = []
    references = []
    reference = []

    result = read_json(prefix+'/result/split_generation_results.json')
    for item in result:
        hypothesis.append(decode_string(item['generation']))
        reference.append(decode_string(item['target']))
        references.append([decode_string(item['target'])])
    
    tokenized_hyps = [x.split() for x in hypothesis]
    tokenized_refs = [[x.split()] for x in reference]
    ngram_match_score = bleu.corpus_bleu(tokenized_refs,tokenized_hyps)

    # calculate weighted ngram match
    keywords = [x.strip() for x in open('keywords/'+lang+'.txt', 'r', encoding='utf-8').readlines()]
    def make_weights(reference_tokens, key_word_list):
        return {token:1 if token in key_word_list else 0.2 \
                for token in reference_tokens}
    tokenized_refs_with_weights = [[[reference_tokens, make_weights(reference_tokens, keywords)]\
                for reference_tokens in reference] for reference in tokenized_refs]

    weighted_ngram_match_score = weighted_ngram_match.corpus_bleu(tokenized_refs_with_weights,tokenized_hyps)

    # calculate syntax match
    syntax_match_score = syntax_match.corpus_syntax_match(references, hypothesis, lang)

    # calculate dataflow match
    dataflow_match_score = dataflow_match.corpus_dataflow_match(references, hypothesis, lang)

    print('ngram match: {0}, weighted ngram match: {1}, syntax_match: {2}, dataflow_match: {3}'.\
                        format(ngram_match_score, weighted_ngram_match_score, syntax_match_score, dataflow_match_score))

    code_bleu_score = alpha*ngram_match_score\
                    + beta*weighted_ngram_match_score\
                    + gamma*syntax_match_score\
                    + theta*dataflow_match_score

    print('CodeBLEU score: ', code_bleu_score)

    
def get_codegpt_codebleu(file_prefix):
    reference = [x for x in open(file_prefix+'test.gold', 'r', encoding='utf-8').readlines()]
    references = [[x] for x in open(file_prefix+'test.gold', 'r', encoding='utf-8').readlines()]
    hypothesis = [x for x in open(file_prefix+'test.output', 'r', encoding='utf-8').readlines()]
    
    tokenized_hyps = [x.split() for x in hypothesis]
    tokenized_refs = [[x.split()] for x in reference]
    ngram_match_score = bleu.corpus_bleu(tokenized_refs,tokenized_hyps)

    # calculate weighted ngram match
    keywords = [x.strip() for x in open('keywords/'+lang+'.txt', 'r', encoding='utf-8').readlines()]
    def make_weights(reference_tokens, key_word_list):
        return {token:1 if token in key_word_list else 0.2 \
                for token in reference_tokens}
    tokenized_refs_with_weights = [[[reference_tokens, make_weights(reference_tokens, keywords)]\
                for reference_tokens in reference] for reference in tokenized_refs]

    weighted_ngram_match_score = weighted_ngram_match.corpus_bleu(tokenized_refs_with_weights,tokenized_hyps)

    # calculate syntax match
    syntax_match_score = syntax_match.corpus_syntax_match(references, hypothesis, lang)

    # calculate dataflow match
    dataflow_match_score = dataflow_match.corpus_dataflow_match(references, hypothesis, lang)

    print('ngram match: {0}, weighted ngram match: {1}, syntax_match: {2}, dataflow_match: {3}'.\
                        format(ngram_match_score, weighted_ngram_match_score, syntax_match_score, dataflow_match_score))

    code_bleu_score = alpha*ngram_match_score\
                    + beta*weighted_ngram_match_score\
                    + gamma*syntax_match_score\
                    + theta*dataflow_match_score

    print('CodeBLEU score: ', code_bleu_score)


In [11]:
def get_metrics_codegpt(prefix):
    
    print('[Rounge-L] Score')
    references = [x for x in open(file_prefix+'test.gold', 'r', encoding='utf-8').readlines()]
    hypothesis = [x for x in open(file_prefix+'test.output', 'r', encoding='utf-8').readlines()]
    result = []
    for ref,hyp in zip(references,hypothesis):
        result.append({'generation':hyp,'target':ref})
    
    
    hyps, refs = map(list, zip(*[[d['generation'], d['target']] for d in result]))
    rouge = Rouge()
    scores = rouge.get_scores(hyps, refs,avg=True)
    print(scores['rouge-l'])
    
    print('[CodeBLEU] Score')
    
    get_codegpt_codebleu(prefix)

file_prefix = '/mnt/jpz_azure_home/save/codegpt_Ori_fix_whitespace_nocolmcc_3_mcct_200_ck_clean_ori_code_mcl_1000//codegpt_adapted_juice/'
get_metrics_codegpt(file_prefix)

[Rounge-L] Score
{'r': 0.20279555654402748, 'p': 0.26721891184061547, 'f': 0.21414256356827652}
[CodeBLEU] Score
ngram match: 0.006786179774655879, weighted ngram match: 0.018638786927896875, syntax_match: 0.18370909570000493, dataflow_match: 0.10819802221950922
CodeBLEU score:  0.06978425120642852


In [49]:
def get_metrics_devdiv(prefix):
    translations_ori = []
    reference_text_ori = []

    print('[BLEU] Score')
    generate_text_path = prefix+'/result/generate-test.txt'
    print(read_txt_last(generate_text_path))
    
    result = read_json(prefix+'/result/split_generation_results.json')
    for item in result:
        translations_ori.append(decode_string(item['generation']))
        reference_text_ori.append(decode_string(item['target']))
    count = 0
    for i,j in zip(translations_ori,reference_text_ori):
        if i==j:
            count += 1
    print('[EM] Score')
    print(len(translations_ori))
    print(count)
    print(count/2000*100)
    
    print('[Rounge-L] Score')
    hyps, refs = map(list, zip(*[[d['generation'], d['target']] for d in result]))
    rouge = Rouge()
    scores = rouge.get_scores(hyps, refs,avg=True)
    print(scores['rouge-l'])
    
    print('[CodeBLEU] Score')
    get_devdiv_codebleu(prefix)

In [56]:
prefix = '/mnt/jpz_azure_home/dataset/juice_filter/Exe_colist_noimcom_nodef_4kdev_resample_mcc_3_mcct_200_ck_clean_ori_code_mcl_1000////'
get_metrics_devdiv(prefix)

[BLEU] Score
P-3513	-2.5666 -2.0677 -0.0000 -0.0212 -0.0023 -0.0004 -0.2398 -0.7950 -0.2773 -0.0011 -0.0031 -0.0001 -0.1817 -1.2833 -0.8154 -0.0000 -0.0000 -0.1386 -0.3908 0.0000 -0.0035 -0.0003 -0.0011 -0.1119 -1.4566 -0.0000 -0.0002 -0.0243 -0.0003 -0.4965 -0.8648 -0.1124 -0.0003 -0.0009 -0.0000 -0.0000 -0.0001 -0.0020 -0.0024 -0.0000 -0.0047 -0.0000 -0.0002 -0.0000 -0.0000 0.0000 -0.0002 0.0000 -0.0000 -0.0000 -0.0000 -0.0000 0.0000 -0.0002 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 0.0000 0.0000 0.0000 -0.0000 0.0000 -0.0000 0.0000 -0.0000 0.0000 -0.0000 -0.0000 -0.0000 0.0000 -0.0000 0.0000 -0.0000 -0.0000 0.0000 -0.0000 0.0000 -0.0000 -0.0000 -0.0000 0.0000 0.0000 -0.0000 -0.0000 0.0000 -0.0000 0.0000 0.0000 -0.0000 0.0000 0.0000 -0.0000 0.0000 0.0000 0.0000 -0.0000 -0.0000 0.0000 -0.0000 -0.0001 0.0000 0.0000 0.0000 -0.0000 0.0000 -0.0000 -0.0000 0.0000 0.0000 -0.0000 0.0000 0.0000 -0.0000 -0.0000 0.0000 0.0000 0.0000 0.0000 -0.0000 0.0000 0.0000 0.0000 0.0000 -0.0000 0.0000 0.0000

In [46]:
prefix = '/mnt/jpz_azure_home/dataset/juice_filter/Ori_fix_whitespace_colist_noimcom_mcc_3_mcct_100_ck_clean_ori_code_mcl_1000//'
result = read_json(prefix+'/result/split_generation_results.json')
translations_ori = []
reference_text_ori = []
for item in result:
    translations_ori.append(decode_string(item['generation']))
    reference_text_ori.append(decode_string(item['target']))
count = 0
for i,j in zip(translations_ori,reference_text_ori):
    print(i)
    print('-'*40)
    print(j)
    break

plt.plot(range(training_epochs),cost_history)
plt.ylabel('jupyter_string')
plt.xlabel('jupyter_string')
plt.show()

plt.plot(range(training_epochs),test_cost)
plt.ylabel('jupyter_string')
plt.xlabel('jupyter_string')
plt.show()

----------------------------------------
plt.plot(range(len(cost_history[1:])),cost_history[1:])
plt.axis([0,training_epochs,0,np.max(cost_history)])
plt.show()


In [40]:
print(result[0]['generation'])

pl t . plot ( range ( training _ ep och s ), cost _ history ) Ċ pl t . yl abel ( madeupword0001 ) Ċ pl t . x label ( madeupword0001 ) Ċ pl t . show () Ċ Ċ pl t . plot ( range ( training _ ep och s ), test _ cost ) Ċ pl t . yl abel ( madeupword0001 ) Ċ pl t . x label ( madeupword0001 ) Ċ pl t . show () Ċ



In [42]:
print(result[0]['target'])

pl t . plot ( range ( len ( cost _ history [ 1 : ]) ), cost _ history [ 1 : ]) Ċ pl t . axis ([ 0 , training _ ep och s , 0 , np . max ( cost _ history ) ]) Ċ pl t . show ()



In [43]:
print(decode_string(result[0]['generation']))

plt.plot(range(training_epochs),cost_history)
plt.ylabel('jupyter_string')
plt.xlabel('jupyter_string')
plt.show()

plt.plot(range(training_epochs),test_cost)
plt.ylabel('jupyter_string')
plt.xlabel('jupyter_string')
plt.show()



In [44]:
print(decode_string(result[0]['target']))

plt.plot(range(len(cost_history[1:])),cost_history[1:])
plt.axis([0,training_epochs,0,np.max(cost_history)])
plt.show()


In [47]:
print(decode_string(result[1]['generation']))

import matplotlib.pyplot as plt
plt.show()
plt.plot(cost_history)


In [48]:
print(decode_string(result[1]['target']))

pred_y = sess.run(y_, feed_dict={X: valid_dataset})
mse = tf.reduce_mean(tf.square(pred_y - valid_labels))
print('jupyter_string' % sess.run(mse)) 

fig, ax = plt.subplots()
ax.scatter(valid_labels, pred_y)
ax.plot([valid_labels.min(), valid_labels.max()], [valid_labels.min(), valid_labels.max()], 'jupyter_string', lw=3)
ax.set_xlabel('jupyter_string')
ax.set_ylabel('jupyter_string')
plt.show()


In [15]:
translations_ori = []
reference_text_ori = []

generate_text_path = prefix+'/result/generate-test.txt'


result = read_json(prefix+'/result/split_generation_results.json')
for item in result:
    translations_ori.append(decode(item['generation']))
    reference_text_ori.append(decode(item['target']))
count = 0
for i,j in zip(translations_ori,reference_text_ori):
    if i==j:
        count += 1
print(len(translations_ori))
print(count)
print(count/2000*100)

1882
157
7.85


In [17]:
hypothesis = []
references = []
reference = []

result = read_json(prefix+'/result/split_generation_results.json')
for item in result:
    hypothesis.append(decode_string(item['generation']))
    reference.append(decode_string(item['target']))
    references.append([decode_string(item['target'])])

In [72]:
print(hypothesis[0])

train_dataset = train_dataset.reshape(train_dataset.shape[0], train_dataset.shape[1], 1)
valid_dataset = valid_dataset.reshape(valid_dataset.shape[0], train_dataset.shape[1], 1)


In [73]:
print(reference[0])

def feature_normalize(dataset):
    mu = np.mean(dataset,axis=0)
    sigma = np.std(dataset,axis=0)
    return (dataset - mu)/sigma
def append_bias_reshape(features,labels = None):
    n_training_samples = features.shape[0]
    n_dim = features.shape[1]
    f = np.reshape(np.c_[np.ones(n_training_samples),features],[n_training_samples,n_dim+1])
    if labels is None:
        return f
    labels = (labels-min(labels))/(max(labels)-min(labels))
    l = np.reshape(labels,[n_training_samples,1])
    return f, l

normalized_train_features = feature_normalize(train_dataset)
normalized_valid_features = feature_normalize(valid_dataset)



train_dataset, train_labels = append_bias_reshape(normalized_train_features,train_labels)
valid_dataset, valid_labels = append_bias_reshape(normalized_valid_features,valid_labels)


train_dataset = np.nan_to_num(train_dataset)
valid_dataset = np.nan_to_num(valid_dataset)



In [22]:
print(references[0])

['data3=pd.read_csv(jupyter_string) \ny=np.asarray(data3.iloc[:,-1]) \nX=np.asarray(data3.iloc[:,0:-1]) ']


In [105]:
file_prefix = '/mnt/jpz_azure_home/save/codegpt_Ori_fix_whitespace_nocolmcc_3_mcct_200_ck_clean_ori_code_mcl_1000//codegpt_adapted_juice/'
reference = [x for x in open(file_prefix+'test.gold', 'r', encoding='utf-8').readlines()]
references = [[x] for x in open(file_prefix+'test.gold', 'r', encoding='utf-8').readlines()]
hypothesis = [x for x in open(file_prefix+'test.output', 'r', encoding='utf-8').readlines()]

In [17]:
tokenized_hyps = [x.split() for x in hypothesis]
tokenized_refs = [[x.split()] for x in reference]

In [18]:
ngram_match_score = bleu.corpus_bleu(tokenized_refs,tokenized_hyps)

# calculate weighted ngram match
keywords = [x.strip() for x in open('keywords/'+lang+'.txt', 'r', encoding='utf-8').readlines()]
def make_weights(reference_tokens, key_word_list):
    return {token:1 if token in key_word_list else 0.2 \
            for token in reference_tokens}
tokenized_refs_with_weights = [[[reference_tokens, make_weights(reference_tokens, keywords)]\
            for reference_tokens in reference] for reference in tokenized_refs]

weighted_ngram_match_score = weighted_ngram_match.corpus_bleu(tokenized_refs_with_weights,tokenized_hyps)

# calculate syntax match
syntax_match_score = syntax_match.corpus_syntax_match(references, hypothesis, lang)

# calculate dataflow match
dataflow_match_score = dataflow_match.corpus_dataflow_match(references, hypothesis, lang)

print('ngram match: {0}, weighted ngram match: {1}, syntax_match: {2}, dataflow_match: {3}'.\
                    format(ngram_match_score, weighted_ngram_match_score, syntax_match_score, dataflow_match_score))

code_bleu_score = alpha*ngram_match_score\
                + beta*weighted_ngram_match_score\
                + gamma*syntax_match_score\
                + theta*dataflow_match_score

print('CodeBLEU score: ', code_bleu_score)

ngram match: 0.10568825685129822, weighted ngram match: 0.10639678937554656, syntax_match: 0.39840607495958796, dataflow_match: 0.2660349127182045
CodeBLEU score:  0.19938211401858907


In [28]:
references = references
candidates = hypothesis
lang = lang

PY_LANGUAGE = Language('/home/v-jipzhang/build/my-languages.so', 'python')
parser = Parser()
parser.set_language(PY_LANGUAGE)
match_count = 0
total_count = 0
out = 0

for i in range(len(candidates)):
    references_sample = references[i]
    candidate = candidates[i] 
    for reference in references_sample:
        try:
            candidate=remove_comments_and_docstrings(candidate,'python')
        except:
            pass    
        try:
            reference=remove_comments_and_docstrings(reference,'python')
        except:
            pass  

        candidate_tree = parser.parse(bytes(candidate,'utf8')).root_node

        reference_tree = parser.parse(bytes(reference,'utf8')).root_node

        def get_all_sub_trees(root_node):
            node_stack = []
            sub_tree_sexp_list = []
            depth = 1
            node_stack.append([root_node, depth])
            while len(node_stack) != 0:
                cur_node, cur_depth = node_stack.pop()
                sub_tree_sexp_list.append([cur_node.sexp(), cur_depth])
                for child_node in cur_node.children:
                    if len(child_node.children) != 0:
                        depth = cur_depth + 1
                        node_stack.append([child_node, depth])
            return sub_tree_sexp_list
        cand_sexps = [x[0] for x in get_all_sub_trees(candidate_tree)]
        ref_sexps = get_all_sub_trees(reference_tree)

        # print(cand_sexps)
        # print(ref_sexps)

        t_match_count = 0
        for sub_tree, depth in ref_sexps:
            if sub_tree in cand_sexps:
                match_count += 1
                t_match_count += 1
        if t_match_count / len(ref_sexps) < 0.3:
            if 'import' not in candidate:
                print(candidate)
                print('='*5)
                print(reference)
                print('-'*40)
            #if 'import' in candidate:
            #    out += 1
print(out)


pca.explained_variance_ratio_
=====
pcaDF = pd.DataFrame(data=pca.components_, index=X.columns)


pcaDF.head()
----------------------------------------
pca1 = PCA(n_components=1)
pca1.fit(X)
X_pca1 = pca1.transform(X)

plt.scatter(X_pca1[:, 0], X_pca1[:, 1])
plt.xlabel('jupyter_string')
plt.ylabel('jupyter_string')
plt.show()
=====
pcaDF[0].plot(kind='jupyter_string', figsize=(15,6))
----------------------------------------
submission_df = pd.read_csv(submission_filename)
submission_df
=====
df1 = pd.read_csv('jupyter_string', 
                  usecols=[0, 1, 2], header=0, names=['jupyter_string', 'jupyter_string', 'jupyter_string'])

df2 = pd.read_csv(submission_filename, 
                  usecols=[0, 1, 2], header=0, names=['jupyter_string', 'jupyter_string', 'jupyter_string'])

df = pd.merge(df1, df2, how='jupyter_string', 
              left_on=['jupyter_string', 'jupyter_string', 'jupyter_string'], 
              right_on=['jupyter_string', 'jupyter_string', 'jupyter_string'])



In [13]:
#print(hypothesis[1])
import ast
def syntax(code):
    try:
        ast.parse(code)
        return True    
    except SyntaxError:
        return False

In [18]:
results = [(code, syntax(code),ref) for code,ref in zip(hypothesis,reference)]
sum([r[1] for r in results])/len(results)

0.9861849096705633

In [19]:
for i,r in enumerate(results):
    if not r[1]:
        print(i)
        print(r[0])
        print('='*5)
        print(r[2])
        print('-'*40)

11
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor


model1 = LinearRegression()
model2 = Lasso()
model3 = Ridge(alpha=0.1)
model4 = ElasticNet(random_state=47)
model5 = RandomForestRegressor(random_state=47)
model6 = BaggingRegressor(max_samples= 0.6, n_estimators= 11, random_state=47)
model7 = GradientBoostingRegressor(random_state=47)
model8 = KNeighborsRegressor(leaf_size= 20, n_neighbors= 4, p= 1)

for i in range(1,17):
    print('jupyter_string'.format(i))

    model1.fit(pcaX_train.iloc[:,:i], y_train)
    model2.fit(pcaX_train.iloc[:,:i], y_train)
    model3.fit(pcaX_train.iloc[:,:i], y_train)
    model4.fit(pcaX_train.iloc[:,:i], y_train)
    model5.fit(pcaX_train.iloc[:,:i], y_train)
    model6.fit(pcaX_train.iloc[:,:i], y_train)
    model7.fit(pcaX_train.iloc[:,:i], y_train)
    model8.fit(pcaX_train.ilo

In [31]:
count = 0
for h, t in zip(hypothesis,reference):
    if syntax(h):
        print(h)
        print('='*5)
        print(t)
        print('-'*20)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.show()

df = pd.read_csv(jupyter_string)
df.head()
=====
data3=pd.read_csv(jupyter_string) 
y=np.asarray(data3.iloc[:,-1]) 
X=np.asarray(data3.iloc[:,0:-1]) 
--------------------
plt.plot(data.Year, data.Crime_Rate)
plt.xlabel(jupyter_string)
plt.ylabel(jupyter_string)
plt.show()
=====
plt.plot(data.index, data[jupyter_string])
--------------------
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)
=====
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(y_test, y_pred))
--------------------
predict_df.to_csv(jupyter_string, index=False)
=====
submission_filename = jupyter_string
predict_df.to_csv(submission_filename, index=False)
--------------------
train_data = pd.read_csv(jupyter_string)
train_labels = pd.read_csv(jupyter_string)
test_data = pd.read_csv(jupyter_string)
=====
X_train = pd.read_csv(jupyter_string)
X_t

In [28]:
for h, t in zip(hypothesis,reference):
    #if h==t:
    print(h)
    print('='*5)
    print(t)
    print('-'*20)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.show()

df = pd.read_csv(jupyter_string)
df.head()
=====
data3=pd.read_csv(jupyter_string) 
y=np.asarray(data3.iloc[:,-1]) 
X=np.asarray(data3.iloc[:,0:-1]) 
--------------------
plt.plot(data.Year, data.Crime_Rate)
plt.xlabel(jupyter_string)
plt.ylabel(jupyter_string)
plt.show()
=====
plt.plot(data.index, data[jupyter_string])
--------------------
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)
=====
from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(y_test, y_pred))
--------------------
predict_df.to_csv(jupyter_string, index=False)
=====
submission_filename = jupyter_string
predict_df.to_csv(submission_filename, index=False)
--------------------
train_data = pd.read_csv(jupyter_string)
train_labels = pd.read_csv(jupyter_string)
test_data = pd.read_csv(jupyter_string)
=====
X_train = pd.read_csv(jupyter_string)
X_t